In [14]:
#dependencies
import numpy as np
import cv2
import pandas as pd
import matplotlib.pyplot as plt
from keras.utils import np_utils
from keras.models import Sequential, Model, model_from_json
from keras.layers import Conv2D, Activation, MaxPool2D, Dropout, Dense, BatchNormalization, Flatten
from keras.callbacks import ModelCheckpoint
import tarfile
import dlib
import time
import os
import re

In [15]:
#0=Angry, 1=Disgust, 2=Fear, 3=Happy, 4=Sad, 5=Surprise, 6=Neutral

In [22]:
def get_landmarks(filename):
    frame = Image.open(filename)
    predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
    detector = dlib.get_frontal_face_detector()
    #frame = cv2.imread(filename,1)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    clahe_image = clahe.apply(frame)
    detections = detector(clahe_image, 1)
    for k,d in enumerate(detections): 
        shape = predictor(clahe_image, d)
        cv2.circle(frame, (shape.part(i).x, shape.part(i).y), 1, (0,0,255), thickness=2) 
    cv2.imshow("image", frame)

In [23]:
import re
def classify_jaffe(filename):
    #filename="sh.HAsd"
    get_landmarks(filename)
    if(re.search(r'(..\.AN)', filename)):
        print(0)
    if(re.search(r'(..\.DI)', filename)):
        print(1)
    elif(re.search(r'(..\.FE)', filename)):
        print(2)
    elif(re.search(r'(..\.HA)', filename)):
        print(3)
    elif(re.search(r'(..\.SA)', filename)):
        print(4)
    elif(re.search(r'(..\.SU)', filename)):
        print(5)
    elif(re.search(r'(..\.NE)', filename)):
        print(6)

In [24]:
o_Dir = "data_landmarks"
dataset_jaffe = "jaffe"
if not os.path.exists(o_Dir):
    print("Making data_landmark directory")
    os.mkdir(o_Dir)
else:
    print("Output directory already exists")
file = []
for (dirpath, dirnames, filenames) in os.walk(dataset_jaffe):
    file.extend(filenames)
    classify_jaffe(str(filenames))
    break

Output directory already exists


RuntimeError: Unsupported image type, must be 8bit gray or RGB image.

In [ ]:
#Set up some required objects
video_capture = cv2.VideoCapture(0) #Webcam object
#detector = dlib.get_frontal_face_detector() #Face detector
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat") #Landmark identifier. Set the filename to whatever you named the downloaded file
###
ap = argparse.ArgumentParser()
ap.add_argument("-c", "--cascade", required=True, help = "haarcascade_frontalface_default.xml")
#ap.add_argument("-p", "--shape-predictor", required=True, help="path to facial landmark predictor")
args = vars(ap.parse_args())

detector = cv2.CascadeClassifier(args["cascade"])
#predictor = dlib.shape_predictor(args["shape_predictor"])

#facec = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

fno=0
detections=()
while True:
    ret, frame = video_capture.read()
    #frame=cv2.imread('a.jpg',1)
    fno+=1;
    if(False):
        pass
    else:
        print(fno)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
        clahe_image = clahe.apply(gray)
        #faces = facec.detectMultiScale(clache_image, 1.3, 5)
        # for (x, y, w, h) in faces:
         #   fc = gray_fr[y:y+h, x:x+w]
          #  roi = cv2.resize(fc, (48, 48))
        detections = detector(clahe_image, 1) #Detect the faces in the image
        #print(detections)
    for k,d in enumerate(detections): #For each detected face
        shape = predictor(clahe_image, d) #Get coordinates
        for i in range(1,68): #There are 68 landmark points on each face
            cv2.circle(frame, (shape.part(i).x, shape.part(i).y), 1, (0,0,255), thickness=2) #For each point, draw a red circle with thickness2 on the original frame
    cv2.imshow("image", frame) #Display the frame
    
    if cv2.waitKey(1) == 27: #Exit program when the user presses 'q'
        break